# **Data from Kaggle in Google Colab using command-line**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir -p /root/.kaggle/

In [ ]:
!cp "/content/drive/MyDrive/Big Data Lab/kaggle.json" /root/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d nelgiriyewithana/global-youtube-statistics-2023

  0% 0.00/60.1k [00:00<?, ?B/s]
100% 60.1k/60.1k [00:00<00:00, 59.3MB/s]


In [ ]:
!unzip global-youtube-statistics-2023.zip

Archive:  global-youtube-statistics-2023.zip
  inflating: Global YouTube Statistics.csv  


# **Required pakages of big data**

In [ ]:
#openjdk installation
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Downloading the apche spark with specific version
!wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
#Zip into unzip the apche spark
!tar -xf spark-3.4.1-bin-hadoop3.tgz
#findspark installation
!pip install -q findspark
#Environment setup for the openjdk and spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# **Youtube Data Analysis**

In [ ]:
df = spark.read.format('csv').option("header","true").option("inferschema","true").option("mode","failfast").load("/content/Global YouTube Statistics.csv")

In [ ]:
df.printSchema()

root
 |-- rank: integer (nullable = true)
 |-- Youtuber: string (nullable = true)
 |-- subscribers: integer (nullable = true)
 |-- video views: double (nullable = true)
 |-- category: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- uploads: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Abbreviation: string (nullable = true)
 |-- channel_type: string (nullable = true)
 |-- video_views_rank: integer (nullable = true)
 |-- country_rank: string (nullable = true)
 |-- channel_type_rank: string (nullable = true)
 |-- video_views_for_the_last_30_days: string (nullable = true)
 |-- lowest_monthly_earnings: double (nullable = true)
 |-- highest_monthly_earnings: double (nullable = true)
 |-- lowest_yearly_earnings: double (nullable = true)
 |-- highest_yearly_earnings: double (nullable = true)
 |-- subscribers_for_last_30_days: string (nullable = true)
 |-- created_year: string (nullable = true)
 |-- created_month: string (nullable = true)
 |-- created

In [ ]:
df.createOrReplaceTempView("yt")

In [ ]:
spark.sql("Select * from yt").show()

+----+--------------------+-----------+---------------+----------------+--------------------+-------+--------------+------------+-------------+----------------+------------+-----------------+--------------------------------+-----------------------+------------------------+----------------------+-----------------------+----------------------------+------------+-------------+------------+---------------------------------------+----------+-----------------+----------------+---------+-----------+
|rank|            Youtuber|subscribers|    video views|        category|               Title|uploads|       Country|Abbreviation| channel_type|video_views_rank|country_rank|channel_type_rank|video_views_for_the_last_30_days|lowest_monthly_earnings|highest_monthly_earnings|lowest_yearly_earnings|highest_yearly_earnings|subscribers_for_last_30_days|created_year|created_month|created_date|Gross tertiary education enrollment (%)|Population|Unemployment rate|Urban_population| Latitude|  Longitude|
+---

In [ ]:
spark.sql("Select count(*) from yt").show()

+--------+
|count(1)|
+--------+
|     995|
+--------+



**Q1: What is the maximum number of YouTube subscribers according to the channel and the name of the channel**

In [ ]:
spark.sql("SELECT MAX(subscribers) AS max_subscribers FROM yt").show()

+---------------+
|max_subscribers|
+---------------+
|      245000000|
+---------------+



In [ ]:
spark.sql("SELECT Youtuber FROM yt WHERE subscribers = (SELECT MAX(subscribers) FROM yt)").show()

+--------+
|Youtuber|
+--------+
|T-Series|
+--------+



**Q2: What is the minimum number of YouTube subscribers according to the channel and the name of the channel**

In [ ]:
spark.sql("SELECT MIN(subscribers) AS min_subscribers FROM yt").show()

+---------------+
|min_subscribers|
+---------------+
|       12300000|
+---------------+



In [ ]:
spark.sql("SELECT Youtuber FROM yt WHERE subscribers = (SELECT MIN(subscribers) FROM yt)").show()

+--------------------+
|            Youtuber|
+--------------------+
|        Natan por A�|
|Free Fire India O...|
|               Panda|
|         RobTopGames|
|        Make Joke Of|
+--------------------+



**Q3: Find out the total number of views of a particular channel according to its name**

In [ ]:
spark.sql("SELECT `video views` FROM yt WHERE Youtuber = 'WWE'").show()

+---------------+
|    video views|
+---------------+
|7.7428473662E10|
+---------------+



In [ ]:
spark.sql("SELECT `video views` FROM yt WHERE Youtuber = 'RobTopGames'").show()

+------------+
| video views|
+------------+
|3.74123483E8|
+------------+



In [ ]:
spark.sql("SELECT `video views` FROM yt WHERE Youtuber = 'T-Series'").show()

+-----------+
|video views|
+-----------+
|    2.28E11|
+-----------+



**Q4: Total number of unique category channels on youtube**

In [ ]:
spark.sql("SELECT COUNT(DISTINCT category) AS unique_category_count FROM yt").show()

+---------------------+
|unique_category_count|
+---------------------+
|                   19|
+---------------------+



**Q5: Find out which category of videos have the most views on YouTube**

In [ ]:
spark.sql("SELECT category, SUM(`video views`) AS total_video_views FROM yt GROUP BY category ORDER BY total_video_views DESC LIMIT 1").show()

+--------+-----------------+
|category|total_video_views|
+--------+-----------------+
|   Music|3.121477506633E12|
+--------+-----------------+



In [ ]:
spark.sql("SELECT category, SUM(`video views`) AS total_video_views FROM yt GROUP BY category ORDER BY total_video_views DESC LIMIT 5").show()

+--------------+-----------------+
|      category|total_video_views|
+--------------+-----------------+
|         Music|3.121477506633E12|
| Entertainment|2.527739309583E12|
|People & Blogs|1.265791201548E12|
|        Gaming| 7.17638899629E11|
|     Education| 6.96614472899E11|
+--------------+-----------------+



**Q6: Find out which category of videos have the less views on YouTube**

In [ ]:
spark.sql("SELECT category, SUM(`video views`) AS total_video_views FROM yt GROUP BY category ORDER BY total_video_views ASC LIMIT 1").show()

+---------------+-----------------+
|       category|total_video_views|
+---------------+-----------------+
|Travel & Events|     3.14088314E9|
+---------------+-----------------+



In [ ]:
spark.sql("SELECT category, SUM(`video views`) AS total_video_views FROM yt GROUP BY category ORDER BY total_video_views ASC LIMIT 5").show()

+--------------------+-----------------+
|            category|total_video_views|
+--------------------+-----------------+
|     Travel & Events|     3.14088314E9|
|Nonprofits & Acti...|  1.0862911785E10|
|    Autos & Vehicles|  1.5003458141E10|
|              Movies|  1.5383255129E10|
|            Trailers|  3.3262717607E10|
+--------------------+-----------------+



**Q7: Find the title value corresponding to the maximum number of views from the video views column from the title column.**

In [ ]:
spark.sql("SELECT Title FROM yt WHERE `video views` = (SELECT MAX(`video views`) FROM yt)").show()

+--------+
|   Title|
+--------+
|T-Series|
+--------+



**Q8: Find the title value corresponding to the minimum number of views from the video views column from the title column.**

In [ ]:
spark.sql("SELECT Title FROM yt WHERE `video views` = (SELECT MIN(`video views`) FROM yt)").show()

+------------------+
|             Title|
+------------------+
|     youtubemovies|
|             Music|
|            Gaming|
|            sports|
|              News|
|Popular on Youtube|
| Minecraft - Topic|
|              Live|
+------------------+



**Q9: Find out which channel has the maximum video uploaded**

In [ ]:
spark.sql("SELECT Youtuber, uploads FROM yt WHERE uploads = (SELECT MAX(uploads) FROM yt)").show()

+--------+-------+
|Youtuber|uploads|
+--------+-------+
|ABP NEWS| 301308|
+--------+-------+



**Q10: Find out which channel has the minimum video uploaded**

In [ ]:
spark.sql("SELECT Youtuber, uploads FROM yt WHERE uploads = (SELECT MIN(uploads) FROM yt)").show()

+--------------------+-------+
|            Youtuber|uploads|
+--------------------+-------+
|               Music|      0|
|              Gaming|      0|
|         BRIGHT SIDE|      0|
|    Luisito Comunica|      0|
|                News|      0|
|T-Series Apna Punjab|      0|
|          Luis Fonsi|      0|
|       Frost Diamond|      0|
|  Aditya Music India|      0|
|  Sandeep Maheshwari|      0|
|1MILLION Dance St...|      0|
|       Fede Vigevani|      0|
|         Chris Brown|      0|
|            FaZe Rug|      0|
|         Alan Becker|      0|
|                YOLO|      0|
|   Minecraft - Topic|      0|
|         Linkin Park|      0|
|      Family GamesTV|      0|
|    Robin Hood Gamer|      0|
+--------------------+-------+
only showing top 20 rows



**Q11: Total number of videos uploaded to YouTube**

In [ ]:
spark.sql("SELECT SUM(uploads) AS total_uploads FROM yt").show()

+-------------+
|total_uploads|
+-------------+
|      9141190|
+-------------+



In [ ]:
spark.sql("Select * from yt").show()

+----+--------------------+-----------+---------------+----------------+--------------------+-------+--------------+------------+-------------+----------------+------------+-----------------+--------------------------------+-----------------------+------------------------+----------------------+-----------------------+----------------------------+------------+-------------+------------+---------------------------------------+----------+-----------------+----------------+---------+-----------+
|rank|            Youtuber|subscribers|    video views|        category|               Title|uploads|       Country|Abbreviation| channel_type|video_views_rank|country_rank|channel_type_rank|video_views_for_the_last_30_days|lowest_monthly_earnings|highest_monthly_earnings|lowest_yearly_earnings|highest_yearly_earnings|subscribers_for_last_30_days|created_year|created_month|created_date|Gross tertiary education enrollment (%)|Population|Unemployment rate|Urban_population| Latitude|  Longitude|
+---

**Q12: Find out number of countries are there channels on YouTube**

In [ ]:
spark.sql("SELECT COUNT(DISTINCT Country) AS unique_country_count FROM yt").show()

+--------------------+
|unique_country_count|
+--------------------+
|                  50|
+--------------------+



**Q13: Which country has the highest number of YouTube channels and the number of channels**

In [ ]:
spark.sql("SELECT Country, COUNT(*) AS count FROM yt GROUP BY Country ORDER BY count DESC LIMIT 1").show()

+-------------+-----+
|      Country|count|
+-------------+-----+
|United States|  313|
+-------------+-----+



**Q14: Which country has the lowest number of YouTube channels and the number of channels**

In [ ]:
spark.sql("SELECT Country, COUNT(*) AS count FROM yt GROUP BY Country ORDER BY count ASC LIMIT 1").show()

+--------+-----+
| Country|count|
+--------+-----+
|Malaysia|    1|
+--------+-----+



**Q15: Total number of unique types of channels on youtube**

In [ ]:
spark.sql("SELECT COUNT(DISTINCT channel_type) AS unique_channel_type_count FROM yt").show()

+-------------------------+
|unique_channel_type_count|
+-------------------------+
|                       15|
+-------------------------+

